# WHO Coronavirus Disease (COVID-19) Dashboard

https://covid19.who.int

- Goto the website
- Download the csv file into a new **local** folder
- Start Jupyter Notebook
- Navigate to the folder
- Open a new Notebook there

## Python text strings

In [ ]:
'https://' 'covid19.who.int'

In [ ]:
who_url = 'https://covid19.who.int'

In [ ]:
who_url

In [ ]:
print?

In [ ]:
print(who_url)

#### Introducing the indexing operator '[ ]'

In [ ]:
who_url[1]

In [ ]:
who_url[-1]

In [ ]:
slice?

In [ ]:
x = slice(5)
x

In [ ]:
who_url[x]

In [ ]:
who_url[:5]

In [ ]:
who_url[-3:-1]

In [ ]:
x = slice(-3, None)
x

In [ ]:
who_url[x]

In [ ]:
who_url[::-1]

In [ ]:
who_url.upper()

#### Introducing the Python list '[ ]'

In [ ]:
l = who_url.split('.')
l

In [ ]:
l = l + ['csv']

In [ ]:
l += [print]
l

In [ ]:
l[-1](l[0])

In [ ]:
csv_file = 'WHO-COVID-19-global-data.csv'

In [ ]:
csv_url = who_url + '/' + csv_file

## Import a 3rd party package:

In [ ]:
import pandas as pd

In [ ]:
dir(pd)

In [ ]:
pd.__version__

In [ ]:
type(dir(pd))

In [ ]:
len(dir(pd))

### What read methods do we have?
_Readability counts._

In [ ]:
range(10)

In [ ]:
list(range(10))

In [ ]:
# C-style loop
dir_pd = dir(pd)
for i in range(len(dir_pd)):
    if dir_pd[i].startswith('read'):
        print(dir_pd[i])

In [ ]:
# Python-style loop
for item in dir(pd):
    if item.startswith('read'):
        print(item)

In [ ]:
x = 1 if 7 < 3 else 2
x

In [ ]:
[x for x in dir(pd) if x.startswith('read')]

In [ ]:
import this

In [ ]:
pd.read_csv?

#### Firewall issues

In [ ]:
# The following code does't work behind a firewall:
# df = pd.read_csv(url)

# Instead use the 'proxifyer' function
import proxify
df = pd.read_csv(proxify.proxify(csv_url))

# Alternatively download the csv-file via a Webbrowser
# and import it directly:
# df = pd.read_csv(csv_file)

## Understanding the DataFrame

In [ ]:
print(df)

In [ ]:
df.info()

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
# take care of the empty spaces in the column names
df.columns = df.columns.str.strip()
df.columns

In [ ]:
df.values

In [ ]:
# surprisingly there are some negative deaths 
# (aka "resurrections") in the data?
df.describe()

### Selection via column attribute

In [ ]:
df.Country

In [ ]:
df.Country.unique()

## Selection via indexing operator '[ ]'  
- select columns by label

In [ ]:
df['Country'].to_frame()

In [ ]:
df[['Country', 'WHO_region']]

- select rows by slice

In [ ]:
df[-3:]

- select rows by bool

In [ ]:
ix = df.Country == 'Germany'
ix

In [ ]:
df[ix]

In [ ]:
df[(df.Country == 'Germany') & (df.New_cases > 5000)]

In [ ]:
df.query('Country == "Germany" and New_cases > 5000')

### With a DataFrame you can't select a row by index <br>using the indexing operator only!
With a Series you can.

In [ ]:
try:
    df[0]
except KeyError as err:
    print('KeyError:', err)

- Select rows and columns by chained indexing

In [ ]:
df[ix].New_cases

In [ ]:
df.New_cases[ix]

### Warning: Don't use chained indexing for assignments!

In [ ]:
df.New_cases[:1] = 7

## Plotting data

In [ ]:
%matplotlib inline

In [ ]:
df[ix].plot()

In [ ]:
df.New_cases[ix].plot(title='New cases Germany', grid=True);

In [ ]:
df.Date_reported = pd.to_datetime(df.Date_reported)

In [ ]:
df.dtypes

In [ ]:
df.Date_reported[0].value

# inplace=True
- inplace, contrary to what the name implies, often does not prevent copies from being created, and (almost) never offers any performance benefits
- inplace does not work with method chaining
- inplace is a common pitfall for beginners, so removing this option will simplify the API

**I don't advise setting this parameter as it serves little purpose.**

```Python
df.set_index(['Date_reported'], inplace=True)
df.sort_index(inplace=True)
```

In [ ]:
df = df.set_index(['Date_reported']).sort_index()
df

In [ ]:
ax = (df
      .New_cases[df.Country=='Germany']
      .plot(title='New cases Germany', grid=True))
ax.set_ylim(0, 8000)

In [ ]:
df[df.Country == 'Germany'].plot(
    x='Cumulative_cases', 
    y='Cumulative_deaths', 
    title='Deaths vs. Infections', grid=True);

### Select rows and columns via loc attribute

In [ ]:
df.loc['2020-01-11', 'Cumulative_cases']

In [ ]:
df.loc['2020-01', 'Cumulative_cases']

### Select rows and columns via iloc attribute

In [ ]:
df.iloc[[1, 3, 7], [1, 2]]

## Grouping rows

In [ ]:
df.groupby('Country').sum()

In [ ]:
df.groupby('Country')[['New_cases', 'Cumulative_cases']].sum()

In [ ]:
sum, max

### Introducing the Python dictionary (dict) '{ }'

In [ ]:
rule = {'New_cases': sum, 'Cumulative_cases': max}
rule

In [ ]:
cases = (df
 .groupby(['Country'])[['New_cases', 'Cumulative_cases']]
 .agg(rule))
cases

In [ ]:
all(cases.New_cases == cases.Cumulative_cases)

In [ ]:
cases[cases.New_cases != cases.Cumulative_cases]

In [ ]:
df[df.New_cases < 0]

### Group by 2 columns

https://en.wikipedia.org/wiki/WHO_regions

In [ ]:
df.WHO_region.unique()

In [ ]:
s = df.groupby(['Date_reported', 'WHO_region'])['New_cases'].sum()
s

### Multiindex Series

In [ ]:
type(s)

In [ ]:
s[:, 'EURO']

In [ ]:
piv = s.unstack('WHO_region')
piv

In [ ]:
piv.loc[:, 'EURO'].dropna().astype(int)

In [ ]:
piv.plot(title='New_cases')

In [ ]:
ax = piv.plot(
    title='New_cases', kind='bar', stacked=True, figsize=(12, 4))
ax.set_xticks([])

In [ ]:
pd.pivot_table(
    df, 
    values='New_cases', 
    index='Date_reported', 
    columns='WHO_region', 
    aggfunc=sum).plot()

In [ ]:
pd.pivot_table(
    df, 
    values='New_deaths', 
    index='Date_reported', 
    columns='WHO_region', 
    aggfunc=sum).plot()

## Add new columns

In [ ]:
df['Mortality'] = df.Cumulative_deaths / df.Cumulative_cases
df

In [ ]:
df.Mortality.max()

In [ ]:
df.Mortality.idxmax()

In [ ]:
df.loc[df.Mortality.idxmax()]

In [ ]:
df.index.is_unique

### Multiindex DataFrame

In [ ]:
df2 = df.set_index('Country', append=True).sort_index()
df2

In [ ]:
df2.index.is_unique

In [ ]:
df2.Mortality.idxmax()

In [ ]:
df2.loc[df2.Mortality.idxmax()]

In [ ]:
df2.loc[(slice(None), 'Germany'), :]  # you have to specify both axes here!

In [ ]:
df2.xs('Germany', level=1)['2020-04']

## Joining 2 Series

Create a plot showing New_cases Germany vs. EURO

In [ ]:
de = df.loc[df.Country == 'Germany', 'New_cases']
de.name = 'New cases Germany'
de

In [ ]:
# 1st option:
eu = (df
      .loc[df.WHO_region == 'EURO', 'New_cases']
      .groupby('Date_reported')
      .sum())
eu.name = 'New cases Europe'      
eu

In [ ]:
# 2nd option
eu = (df
      .groupby(['Date_reported', 'WHO_region'])
      .sum()
      .loc[(slice(None), 'EURO'), 'New_cases']
      .droplevel('WHO_region'))
eu.name = 'New cases Europe'     
eu

In [ ]:
pd.concat([de, eu], axis=1).plot()

In [ ]:
pd.concat([de, eu], axis=1).rolling(window=7).mean().plot()

## Exercises

1. How many Corona cases have been registered in 2020-Q1 world-wide?

In [ ]:
# your code here ...

In [ ]:
%load solutions/covid_01.py

2. What was the average number of new cases in Germany over the past 30 days?

In [ ]:
# your code here ...

In [ ]:
%load solutions/covid_02.py

3. Print: "The number of Corona deaths in the first half of this year for Sweden was xxxx."  
_Hint: Check thr print() arguments and the df.values attribute._

In [ ]:
# your code here ...

In [ ]:
%load solutions/covid_03.py

4. What was the average number of new cases in Europe over the past 30 days?

In [ ]:
# your code here ...

In [ ]:
%load solutions/covid_04.py

5. Make a 'pie' chart from the average number of new cases per WHO region over the past 30 days.  
_Hint: df.plot(kind='pie')_

In [ ]:
# your code here ...

In [ ]:
%load solutions/covid_05.py